## 1. Weak Labeling with Autoencoder

30 labeled, 300 unlabeled

In [ ]:
get_objects to randomly set aside 30 LTr, 300 ULTr, and 70 LTe objects

tube_loader to get frames for 330 *Tr objects for Reef

pass train_primitive_matrix into weak_label

## 2. Create Action Tubes from Weakly-Labeled Frame